In [1]:
import sys
import os.path as osp
sys.path
sys.path.append('../../L1DeepMETv2/')
from graphmetnetwork import GraphMetNetwork

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.utils import to_undirected
from torch_cluster import radius_graph, knn_graph
from torch_geometric.datasets import MNISTSuperpixels
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from tqdm import tqdm
import model.net as net
import model.data_loader as data_loader
import utils

/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Parameters

In [2]:
weights = '../weights/'
data_dir = '../../L1DeepMETv2/data_ttbar'

In [3]:
dataloaders = data_loader.fetch_dataloader(data_dir = data_dir, batch_size=6, validation_split=.2)
train_dl = dataloaders['train']
test_dl = dataloaders['test']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Training dataloader: {}, Test dataloader: {}'.format(len(train_dl), len(test_dl)))
print(device)

Processing...
0it [00:00, ?it/s]
Done!


Split: 199708
length of train/val data:  798834 199708
Training dataloader: 133139, Test dataloader: 33285
cuda


/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [4]:
test_data = None
for cnt, test_data in enumerate(test_dl):
    if cnt == 5:
        break

#### Load Tensor Parameters

In [5]:
n_features_cont = 6
x_cont_test = test_data.x[:,:n_features_cont] .to(device)  # include puppi
x_cat_test = test_data.x[:,n_features_cont:].long().to(device)
etaphi_test = torch.cat([test_data.x[:, 3][:, None], test_data.x[:, 4][:, None]], dim=1).to(device=device)
batch_test = test_data.batch.to(device)
edge_index_test = radius_graph(etaphi_test, r=0.4, batch=batch_test, loop=False, max_num_neighbors=255).to(device=device)
print(f'x_cont_test: {x_cont_test.shape}')
print(f'x_cat_test: {x_cat_test.shape}')
print(f'etaphi: {etaphi_test.shape}')
print(f'batch: {batch_test.shape}')
print(f'edge_index: {edge_index_test.shape}')

/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/torch/cuda/__init__.py:146: UserWarning: 
NVIDIA RTX A6000 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA RTX A6000 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


x_cont_test: torch.Size([267, 6])
x_cat_test: torch.Size([267, 2])
etaphi: torch.Size([267, 2])
batch: torch.Size([267])
edge_index: torch.Size([2, 1494])


#### Convert Tensor parameters to Numpy arrays

In [6]:
x_cont = np.ascontiguousarray(x_cont_test.squeeze(0).cpu().numpy())
x_cat = np.ascontiguousarray(x_cat_test.squeeze(0).cpu().numpy())
batch = np.ascontiguousarray(batch_test.squeeze(0).cpu().numpy())
etaphi = etaphi_test.squeeze(0).cpu().numpy()
edge_index = edge_index_test.squeeze(0).cpu().numpy().transpose()
num_nodes = x_cont.shape[0]
batch_size = batch.shape[0]
print(f'Number of nodes: {num_nodes}')
assert(num_nodes == batch_size)

Number of nodes: 267


### Load the Torch Model

In [7]:
prefix = '../../L1DeepMETv2/ckpts_April30_scale_sigmoid'
# Restore ckpts
restore_ckpt = osp.join(prefix, 'last.pth.tar')
norm = torch.tensor([1., 1., 1., 1., 1., 1.]).to(device=device)
torch_model = net.Net(continuous_dim=6, categorical_dim=2 , norm=norm).to(device)
print(torch_model)

Net(
  (graphnet): GraphMETNetwork(
    (embed_charge): Embedding(3, 8)
    (embed_pdgid): Embedding(7, 8)
    (embed_continuous): Sequential(
      (0): Linear(in_features=6, out_features=16, bias=True)
      (1): ELU(alpha=1.0)
    )
    (embed_categorical): Sequential(
      (0): Linear(in_features=16, out_features=16, bias=True)
      (1): ELU(alpha=1.0)
    )
    (encode_all): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): ELU(alpha=1.0)
    )
    (bn_all): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_continuous): ModuleList(
      (0): ModuleList(
        (0): EdgeConv(nn=Sequential(
          (0): Linear(in_features=64, out_features=32, bias=True)
        ))
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ModuleList(
        (0): EdgeConv(nn=Sequential(
          (0): Linear(in_features=64, out_features=32, bias=True)
        ))
     

/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:988: UserWarning: 'EdgeConv.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "


In [8]:
param_restored_new = utils.load_checkpoint(restore_ckpt, torch_model)
param_restored_new

{'epoch': 62,
 'state_dict': OrderedDict([('graphnet.embed_charge.weight',
               tensor([[ 0.9152, -1.0589, -0.9910,  0.0120, -1.2543,  0.2998, -0.3684, -0.0260],
                       [ 1.1699, -1.1929, -0.4268, -0.7047, -0.3582,  0.5368,  1.0060, -0.7781],
                       [-1.1897, -0.7680,  0.9429,  0.2915, -0.2274, -1.3632,  0.6982,  0.4960]],
                      device='cuda:0')),
              ('graphnet.embed_pdgid.weight',
               tensor([[-2.0383, -0.3847, -0.2413,  0.9122, -0.4805,  0.2302, -0.4746,  0.9317],
                       [-0.4117,  1.3556,  0.8550,  0.9977, -0.1622,  1.0582, -0.3220, -0.1858],
                       [ 0.3344, -1.7237,  0.2500,  0.0249, -1.0838, -0.7614, -0.2512, -1.4239],
                       [-0.0926,  0.4536, -0.2328,  1.5914, -0.3767, -1.6118,  0.4281,  0.3517],
                       [-0.8126, -1.1399, -0.2381, -0.4232,  0.0113, -1.9791, -0.4646, -1.3267],
                       [ 0.1623,  0.1171, -0.4400,  0.8133, -

### Load the C++ Model

In [9]:
# Create an instance of the C++ GraphMetNetwork model
cmodel = GraphMetNetwork()

# Load the weights
cmodel.load_weights(weights)

### Test the weights

In [12]:
torch_emb_cont_weights = param_restored_new['state_dict']['graphnet.embed_continuous.0.weight'].cpu().numpy()
torch_emb_cont_weights.shape

(16, 6)

In [14]:
cmodel_emb_cont_weights = cmodel.get_graphmet_embed_continuous_0_weight()
cmodel_emb_cont_weights.shape

(16, 6)

In [16]:
print(torch_emb_cont_weights)

[[-0.51976335  0.29137233 -0.01950876  0.3422203  -0.22453895  0.18464667]
 [-0.41697326 -0.19378653 -0.2800268  -0.03558999 -0.16028096  0.1395752 ]
 [-0.1805233  -0.11106941 -0.23513117 -0.11494771 -0.13471612 -0.46705556]
 [-0.09102532  0.03195464 -0.11112893  0.07975572  0.18790151  0.36718494]
 [-0.63014483 -0.1793874  -0.166681   -0.14576283  0.20136194 -0.3199493 ]
 [ 0.3729181  -0.03066996  0.02352745 -0.00087973  0.23562688 -0.4000545 ]
 [ 0.47255328 -0.05097763 -0.1412205   0.17181246 -0.11328534  0.17674764]
 [ 0.04552776 -0.01817972  0.04870505  0.19884565  0.23553249  0.29376933]
 [-0.45972535 -0.18024738 -0.37173915 -0.14721555 -0.26793975  0.32831663]
 [ 0.12318032 -0.33687368 -0.20999411 -0.17968899 -0.00177267 -0.00941435]
 [ 0.17154491 -0.14698578  0.14260884  0.03123299  0.25386858  0.00154118]
 [-0.3558818   0.091422    0.35282034  0.03867797 -0.09918415 -0.10135012]
 [ 0.06261277 -0.33330274 -0.33637494  0.2582759  -0.22359034  0.30760753]
 [-0.33064967  0.05518911

In [17]:
print(cmodel_emb_cont_weights)

[[-0.01408479  0.26224014  0.21124163 -0.02148151 -0.27710617 -0.18009436]
 [-0.11469425 -0.16827478  0.25944638  0.03087053  0.10644519 -0.3335978 ]
 [-0.2733658   0.1272415  -0.26642194 -0.09995878  0.01028038  0.322061  ]
 [-0.15765736  0.17464443 -0.2051494   0.11367632 -0.29654217  0.11224843]
 [-0.24401419 -0.16353707  0.2782284  -0.01478     0.17794731 -0.11090042]
 [-0.2502501  -0.15644376  0.28004238 -0.02011693  0.21282925 -0.12474898]
 [-0.2524805  -0.16291493  0.28042853  0.01157601  0.21667488 -0.11200128]
 [-0.20589809  0.10273653  0.06480501  0.11469385  0.16831794  0.02854833]
 [-0.31383845  0.13041887 -0.1979931   0.58252996 -0.02516119  0.156358  ]
 [-0.22235836  0.1070975   0.10644051  0.01248249  0.1654896  -0.20216064]
 [-0.29422224 -0.03631451  0.04636873  0.06164292  0.26499832  0.6169511 ]
 [-0.2752517  -0.00498366  0.08971923 -0.14684242 -0.12780145  0.62608856]
 [ 0.11506677 -0.57081324 -0.34112072  0.00434716  0.07189704 -0.086274  ]
 [-0.317476    0.11207171

In [15]:
assert(np.allclose(torch_emb_cont_weights, cmodel_emb_cont_weights, atol=1e-5))

AssertionError: 

### Run the Torch Model

In [ ]:
results = torch_model(x_cont_test, x_cat_test, edge_index_test, batch_test)

### Run the C++ Model

In [ ]:
cmodel.GraphMetNetworkLayers(x_cont, x_cat, batch, num_nodes)

#### Test Inputs

In [ ]:
c_x_cont = cmodel.get_x_cont()
c_x_cat = cmodel.get_x_cat()
c_batch = cmodel.get_batch()
c_num_nodes = cmodel.get_num_nodes()
print(f'Shape of c_x_cont: {c_x_cont.shape}')
print(f'Shape of c_x_cat: {c_x_cat.shape}')
print(f'Shape of c_batch: {c_batch.shape}')
print(f'Value of c_num_nodes: {c_num_nodes}')

In [ ]:
# print(x_cont)
assert(np.allclose(x_cont, c_x_cont, atol=1e-5))
assert(np.allclose(x_cat, c_x_cat, atol=1e-5))
assert(np.allclose(batch, c_batch, atol=1e-5))
assert(np.allclose(num_nodes, c_num_nodes, atol=1e-5))



#### Test Internal Variables

In [ ]:
c_etaphi = cmodel.get_etaphi()
print(etaphi.shape)
print(c_etaphi.shape)
print(type(etaphi))
print(type(c_etaphi))
are_almost_equal = np.allclose(etaphi, c_etaphi, atol=1e-5)
assert(np.allclose(etaphi, c_etaphi, atol=1e-5))

In [ ]:
c_num_edges = cmodel.get_num_edges()
c_edge_index = cmodel.get_edge_index()
edge_index_np = edge_index_test.squeeze(0).cpu().numpy().transpose()
print(edge_index_np.shape)
print(c_edge_index.shape)
print(f'Number of C edges: {c_num_edges}')

In [ ]:
print(edge_index_np)
print(c_edge_index)

#### Test Intermediate Variables

In [ ]:
torch_emb_cont = torch_model.graphnet.emb_cont_.cpu().numpy()
cmodel_emb_cont = cmodel.get_emb_cont()
# np.testing.assert_allclose(torch_model.graphnet.emb_cont_.cpu().numpy(), cmodel.get_emb_cont(), rtol=1e-5)
print(torch_emb_cont.shape)
print(cmodel_emb_cont.shape)

In [ ]:
print(torch_emb_cont)

In [ ]:
print(cmodel_emb_cont)

### Other Tests

In [ ]:
import c_radius_graph

# Example points in 2D space and their corresponding batch indices
points = [[0.0, 0.0], [1.0, 1.0], [2.0, 2.0],  # Batch 0
          [3.0, 3.0], [4.0, 4.0],              # Batch 1
          [5.0, 5.0], [6.0, 6.0]]              # Batch 2
batch_indices = [0, 0, 0, 1, 1, 2, 2]  # Batch assignments
radius = 1.5

In [ ]:
# Call the C++ function
neighbors = c_radius_graph.find_neighbors_by_batch(points, batch_indices, radius)

# Print neighbors
for pair in neighbors:
    print(f"Point {pair[0]} is within radius of point {pair[1]}")

neighbors

In [ ]:
points_tensor = torch.tensor(points)
batch_tensor = torch.tensor(batch_indices)

edge_index_pts = radius_graph(points_tensor, r=radius, batch=batch_tensor, loop=False, max_num_neighbors=255)

In [ ]:
print(edge_index_pts)

In [ ]:
# Example usage in Python
import graphmetnetwork_bindings as gmn

# Create an instance of the model
model = gmn.GraphMetNetwork()

# Load the weights
model.load_weights("path_to_weights_file.txt")

# Now, you can run the model with input data
for i, (x_cont, x_cat, edge_index, batch) in enumerate(dataloader):
    num_nodes = x_cont.shape[0]

    # Run the PyTorch model
    with torch.no_grad():
        output = torch_model(x_cont.squeeze(0), x_cat.squeeze(0), edge_index.squeeze(0), batch.squeeze(0))

    # Run the C++ model
    model.GraphMetNetworkLayer(x_cont.squeeze(0).numpy(), x_cat.squeeze(0).numpy(), num_nodes, batch.squeeze(0).numpy())

    # Compare intermediate values as before
    np.testing.assert_allclose(torch_model._emb_cont.numpy(), model.get_emb_cont(), rtol=1e-5)
    np.testing.assert_allclose(torch_model._emb_chrg.numpy(), model.get_emb_chrg(), rtol=1e-5)
    np.testing.assert_allclose(torch_model._emb_pdg.numpy(), model.get_emb_pdg(), rtol=1e-5)
    np.testing.assert_allclose(torch_model._emb_cat.numpy(), model.get_emb_cat(), rtol=1e-5)
    np.testing.assert_allclose(torch_model._emb.numpy(), model.get_emb(), rtol=1e-5)
    np.testing.assert_allclose(torch_model._emb1.numpy(), model.get_emb1(), rtol=1e-5)
    np.testing.assert_allclose(torch_model._emb2.numpy(), model.get_emb2(), rtol=1e-5)
    np.testing.assert_allclose(output.numpy(), model.get_output(), rtol=1e-5)
